In [2]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets
!pip install transformers
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torchaudio==0.10.0+cu113 (from versions: 0.11.0, 0.11.0+cu113, 0.12.0, 0.12.0+cu113, 0.12.1, 0.12.1+cu113, 0.13.0, 0.13.1, 2.0.0, 2.0.1, 2.0.2, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1)
ERROR: No matching distribution found for torchaudio==0.10.0+cu113


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset, load_metric, Audio
common_voice_train = load_dataset("mozilla-foundation/common_voice_17_0", "ne-NP", split="validated+other", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2554: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
# This cell basically splits the dataset into three subsets. 80% goes to the training set, 10% goes to the validation set and 10% goes to the test set
from datasets import load_dataset

# Define proportions for training, validation, and test sets
train_ratio = 0.8  # 80% for training
validation_ratio = 0.1  # 10% for validation
test_ratio = 0.1  # 10% for testing
common_voice_train = load_dataset("mozilla-foundation/common_voice_17_0", "ne-NP", split="validated+other", use_auth_token=True)
# Calculate sizes based on ratios
num_samples = len(common_voice_train)
print(f'Original: {num_samples}')
train_size = int(train_ratio * num_samples)
validation_size = int(validation_ratio * num_samples)
test_size = num_samples - train_size - validation_size

# Split the dataset
train_dataset = common_voice_train.select(range(0, train_size))
common_voice_dev = common_voice_train.select(range(train_size, train_size + validation_size))
common_voice_test = common_voice_train.select(range(train_size + validation_size, train_size + validation_size + test_size))

common_voice_train = train_dataset

# Print the sizes of each dataset
print(f"Training dataset size: {len(common_voice_train)} samples")
print(f"Validation dataset size: {len(common_voice_dev)} samples")
print(f"Test dataset size: {len(common_voice_test)} samples")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2554: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Original: 1337
Training dataset size: 1069 samples
Validation dataset size: 133 samples
Test dataset size: 135 samples


In [6]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voiec_dev = common_voice_dev.remove_columns(['accent', 'age', 'client_id', 'down_votes', 'gender', 'locale', 'segment', 'up_votes'])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [7]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [8]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_dev = common_voice_dev.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

In [191]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("/content/drive/MyDrive/Nepali_ASR/results/checkpoint-800", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [192]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [193]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [12]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_dev = common_voice_dev.cast_column("audio", Audio(sampling_rate=16_000))

In [13]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

निद्रा त महत्वपूर्ण कुरा हो नि रोशन।


In [14]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]['sentence'])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: हामिले मागेको खण्डमा उपलब्ध गराईन्छ होला त तथ्यांक
Input array shape: (77760,)
Sampling rate: 16000


In [15]:
common_voice_test[0]['audio']['sampling_rate']

16000

In [16]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [17]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_dev = common_voice_dev.map(prepare_dataset, remove_columns=common_voice_dev.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

In [194]:
from transformers import Wav2Vec2ForCTC
model = Wav2Vec2ForCTC.from_pretrained('/content/drive/MyDrive/Nepali_ASR/results/checkpoint-800/')

In [50]:
# Define proportions for training, validation, and test sets
train_ratio = 0.8  # 80% for training
validation_ratio = 0.1  # 10% for validation
test_ratio = 0.1  # 10% for testing
common_voice_train_evaluation = load_dataset("mozilla-foundation/common_voice_17_0", "ne-NP", split="validated+other", use_auth_token=True)
# Calculate sizes based on ratios
num_samples = len(common_voice_train_evaluation)
print(f'Original: {num_samples}')
train_size = int(train_ratio * num_samples)
validation_size = int(validation_ratio * num_samples)
test_size = num_samples - train_size - validation_size

# Split the dataset
train_dataset = common_voice_train_evaluation.select(range(0, train_size))
common_voice_dev_evaluation = common_voice_train_evaluation.select(range(train_size, train_size + validation_size))
common_voice_test_evaluation = common_voice_train_evaluation.select(range(train_size + validation_size, train_size + validation_size + test_size))

common_voice_train_evaluation = train_dataset

# Print the sizes of each dataset
print(f"Training dataset size: {len(common_voice_train_evaluation)} samples")
print(f"Validation dataset size: {len(common_voice_dev_evaluation)} samples")
print(f"Test dataset size: {len(common_voice_test_evaluation)} samples")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2554: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Original: 1337
Training dataset size: 1069 samples
Validation dataset size: 133 samples
Test dataset size: 135 samples


In [53]:
common_voice_train_evaluation = common_voice_train_evaluation.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test_evaluation = common_voice_test_evaluation.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_dev_evaluation = common_voice_dev_evaluation.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
import torch
import random


In [195]:

random_int = random.randint(0, len(common_voice_dev)-1)
input_dict = processor(common_voice_dev[random_int]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [196]:
import IPython.display as ipd

print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_dev_evaluation[random_int]["sentence"].lower())
ipd.Audio(data=common_voice_dev_evaluation[random_int]['audio']['array'], autoplay=True, rate=16000)


Prediction:
मैले हेरे दाइ हिजोको खेल ।

Reference:
मैले हेरे दाइ हिजोको खेल ।


In [79]:
print('Hello World! Thank You!!')

Hello World! Thank You!!


# Inference From a File

In [197]:
from IPython.display import Audio
from scipy.io import wavfile
import numpy as np

In [198]:
from transformers import Wav2Vec2ForCTC

# tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("/content/drive/MyDrive/Nepali_ASR/results/checkpoint-800", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
model = Wav2Vec2ForCTC.from_pretrained('/content/drive/MyDrive/Nepali_ASR/results/checkpoint-800/')

In [199]:
from transformers import Wav2Vec2Tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("/content/drive/MyDrive/Nepali_ASR/results/checkpoint-800", unk_token='[UNK]', pad_token='[PAD]', word_delimiter_token='|')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:720: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(


In [200]:
file_name = '/content/drive/MyDrive/Nepali_ASR/test-3.mp3'

In [201]:
from pydub import AudioSegment

# Load MP3 file
audio = AudioSegment.from_mp3(file_name)

# Export as WAV
wav_file = "/content/drive/MyDrive/Nepali_ASR/test-3-wav.wav"
audio.export(wav_file, format="wav")


<_io.BufferedRandom name='/content/drive/MyDrive/Nepali_ASR/test-3-wav.wav'>

In [202]:
Audio(wav_file)

In [203]:
data = wavfile.read(wav_file)
framerate = data[0]
sounddata = data[1]
time = np.arange(0,len(sounddata))/framerate
print('Sample rate:',framerate,'Hz')
print('Total time:',len(sounddata)/framerate,'s')


Sample rate: 48000 Hz
Total time: 4.056 s


In [204]:
import soundfile as sf
import librosa

In [205]:
input_audio, _ = librosa.load(wav_file,
                              sr=16000)

In [206]:
# input_values = tokenizer(input_audio, return_tensors="pt", padding=True).input_values
input_values = tokenizer(input_audio, return_tensors='pt').input_values
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids)[0]

In [207]:
print(f'Prediction:\n{transcription}')

Prediction:
म अहिले मैजार र्जेर बोति मोती राखेको छु।
